In [27]:
import requests
import pandas as pd
import datetime
from datetime import datetime, timezone

def to_epoch(my_date):
    my_date_dt = datetime.strptime(my_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
    epoch_timestamp = my_date_dt.timestamp()
    return int(epoch_timestamp)

def tick_to_price(tick, decimals0, decimals1):
    return 1.0001 ** - tick * (10 ** (decimals1 - decimals0))

def fetch_uniswap_swaps(pool_address, start_date, end_date, limit=1000):
    endpoint = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"
    query = """
    query MyQuery($pool: String!, $first: Int!, $end_date: Int!, $skip: Int!) {
      swaps(
        where: {pool: $pool, transaction_: {timestamp_lte: $end_date}}
        orderBy: timestamp
        orderDirection: desc
        first: $first
        skip: $skip
      ) {
        amount0
        amount1
        amountUSD
        origin
        tick
        sqrtPriceX96
        sender
        recipient
        timestamp
        recipient
        id
        sqrtPriceX96
        tick
        token1 {
          decimals
          symbol
        }
        token0 {
          decimals
          symbol
        }
        transaction {
          gasUsed
          gasPrice
          blockNumber
          timestamp
        }
      }
    }
    """

    swaps = []
    skip = 0

    variables = {"pool": pool_address, "end_date": end_date, "first": limit, "skip": skip}
    response = requests.post(endpoint, json={'query': query, 'variables': variables})
    try:
        data = response.json()['data']['swaps']
    except:
        pass
        
    df = pd.DataFrame(data)
    print(datetime.utcfromtimestamp(int(df.timestamp.iloc[-1])))
    
    df[['amount0', 'amount1', 'amountUSD', 'timestamp']] = df[['amount0', 'amount1', 'amountUSD', 'timestamp']].astype('float')
    df[['sender', 'recipient', 'origin']] = df[['sender', 'recipient', 'origin']].astype('str')

    df['gasUsed']  = df['transaction'].apply(lambda x: x['gasUsed']).astype('int')
    #df['gasPrice_gwei'] = df['transaction'].apply(lambda x: x['gasPrice']).astype('int')
    #df['gasPrice_eth'] = df['gasPrice_gwei']*10**-18
    df['gasPrice_wei'] = df['transaction'].apply(lambda x: float(x['gasPrice']))
    df['gasPrice_eth'] = df['gasPrice_wei'] * 10**-18
    
    df['blockNumber'] = df['transaction'].apply(lambda x: x['blockNumber']).astype('int')
    #df['timestamp'] = df['transaction'].apply(lambda x: x['timestamp']).astype('int')
    
    df['symbol0'] = df['token0'].apply(lambda x: x['symbol']).astype('str')
    df['symbol1'] = df['token1'].apply(lambda x: x['symbol']).astype('str')

    df['decimal0'] = df['token0'].apply(lambda x: x['decimals']).astype('int')
    df['decimal1'] = df['token1'].apply(lambda x: x['decimals']).astype('int')

    df['tick']  = df['tick'].astype('int')
    df['price'] = df.apply(lambda row: tick_to_price(row['tick'], row['decimal0'], row['decimal1']), axis=1)

    df['tcost_usd'] = df['gasPrice_eth']*df['price']*df['gasUsed']
    
    df['time'] = pd.to_datetime(df['timestamp'], unit='s')

    df = df.drop(['token1', 'token0', 'tick', 'decimal0', 'decimal1', 'id', 'sqrtPriceX96', 'transaction'], axis=1)

    return df

In [28]:
pool_address = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
pool_address = pool_address.lower()

start_date = "2024-01-01 00:00:00"
start_date = to_epoch(start_date)
end_date = "2024-01-31 23:59:59"
end_date = to_epoch(end_date)


In [29]:
# Getting the data from TheGraph and generating the df dataframe
df = fetch_uniswap_swaps(pool_address, start_date, end_date)
while df['timestamp'].iloc[-1]>start_date:
    df = pd.concat([df, fetch_uniswap_swaps(pool_address, start_date, int(df['timestamp'].iloc[-1]-1))], ignore_index=True)
df

df.to_pickle("data_UV3_WETH_USDC_January2024.pkl") #Saving the data locally

2024-01-31 20:57:11
2024-01-31 17:12:35
2024-01-31 13:33:35
2024-01-31 08:56:47
2024-01-31 04:03:11
2024-01-31 00:00:47
2024-01-30 20:03:23
2024-01-30 16:09:47
2024-01-30 11:43:59
2024-01-30 06:21:47
2024-01-30 01:57:59
2024-01-29 22:05:59
2024-01-29 18:25:59
2024-01-29 15:01:59
2024-01-29 11:07:23
2024-01-29 06:42:23
2024-01-29 02:29:11
2024-01-28 22:40:59
2024-01-28 19:19:11
2024-01-28 16:06:23
2024-01-28 10:57:47
2024-01-28 05:07:11
2024-01-28 00:31:47
2024-01-27 19:52:47
2024-01-27 15:35:35
2024-01-27 10:30:35
2024-01-27 08:16:23
2024-01-27 05:57:35
2024-01-27 00:38:47
2024-01-26 20:18:11
2024-01-26 16:44:23
2024-01-26 13:12:59
2024-01-26 09:35:11
2024-01-26 04:50:23
2024-01-26 01:00:35
2024-01-25 21:02:59
2024-01-25 17:20:23
2024-01-25 14:49:59
2024-01-25 10:49:47
2024-01-25 06:57:59
2024-01-25 02:56:59
2024-01-25 00:11:23
2024-01-24 21:18:23
2024-01-24 18:14:11
2024-01-24 14:24:35
2024-01-24 10:17:59
2024-01-24 06:09:23
2024-01-24 02:05:11
2024-01-23 22:56:11
2024-01-23 19:54:47
